In [40]:
!pip install datasets sentence-transformers faiss-cpu openai


In [41]:
import pandas as pd


In [61]:
df=pd.read_excel("/content/Abirate english quotes.xlsx")

In [62]:
df

,quote,author,tags
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[\n ""be-yourself"",\n ""gilbert-perreira"",\n ..."
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[\n ""best"",\n ""life"",\n ""love"",\n ""mistake..."
2,“Two things are infinite: the universe and hum...,Albert Einstein,"[\n ""human-nature"",\n ""humor"",\n ""infinity""..."
3,"“So many books, so little time.”",Frank Zappa,"[\n ""books"",\n ""humor""\n]"
4,“A room without books is like a body without a...,Marcus Tullius Cicero,"[\n ""books"",\n ""simile"",\n ""soul""\n]"
...,...,...,...
95,“I'm not afraid of death; I just don't want to...,Woody Allen,"[\n ""death"",\n ""fear"",\n ""humor"",\n ""life""\n]"
96,“If one cannot enjoy reading a book over and o...,Oscar Wilde,"[\n ""books"",\n ""reading""\n]"
97,"“If you want your children to be intelligent, ...",Albert Einstein,"[\n ""children"",\n ""fairy-tales""\n]"
98,“Have you fallen in love with the wrong person...,"Cassandra Clare,","[\n ""city-of-bones"",\n ""jace""\n]"


In [63]:
df['tags'] = df['tags'].str.replace(r'\n', '', regex=True)
df

,quote,author,tags
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[ ""be-yourself"", ""gilbert-perreira"", ""hones..."
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[ ""best"", ""life"", ""love"", ""mistakes"", ""ou..."
2,“Two things are infinite: the universe and hum...,Albert Einstein,"[ ""human-nature"", ""humor"", ""infinity"", ""ph..."
3,"“So many books, so little time.”",Frank Zappa,"[ ""books"", ""humor""]"
4,“A room without books is like a body without a...,Marcus Tullius Cicero,"[ ""books"", ""simile"", ""soul""]"
...,...,...,...
95,“I'm not afraid of death; I just don't want to...,Woody Allen,"[ ""death"", ""fear"", ""humor"", ""life""]"
96,“If one cannot enjoy reading a book over and o...,Oscar Wilde,"[ ""books"", ""reading""]"
97,"“If you want your children to be intelligent, ...",Albert Einstein,"[ ""children"", ""fairy-tales""]"
98,“Have you fallen in love with the wrong person...,"Cassandra Clare,","[ ""city-of-bones"", ""jace""]"


In [64]:
df.dropna(subset=['quote', 'author'], inplace=True)


In [65]:
df

,quote,author,tags
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[ ""be-yourself"", ""gilbert-perreira"", ""hones..."
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[ ""best"", ""life"", ""love"", ""mistakes"", ""ou..."
2,“Two things are infinite: the universe and hum...,Albert Einstein,"[ ""human-nature"", ""humor"", ""infinity"", ""ph..."
3,"“So many books, so little time.”",Frank Zappa,"[ ""books"", ""humor""]"
4,“A room without books is like a body without a...,Marcus Tullius Cicero,"[ ""books"", ""simile"", ""soul""]"
...,...,...,...
95,“I'm not afraid of death; I just don't want to...,Woody Allen,"[ ""death"", ""fear"", ""humor"", ""life""]"
96,“If one cannot enjoy reading a book over and o...,Oscar Wilde,"[ ""books"", ""reading""]"
97,"“If you want your children to be intelligent, ...",Albert Einstein,"[ ""children"", ""fairy-tales""]"
98,“Have you fallen in love with the wrong person...,"Cassandra Clare,","[ ""city-of-bones"", ""jace""]"


In [66]:
df=df.drop_duplicates()

In [67]:
df=df.reset_index(drop=True)

In [68]:
df

,quote,author,tags
0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"[ ""be-yourself"", ""gilbert-perreira"", ""hones..."
1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"[ ""best"", ""life"", ""love"", ""mistakes"", ""ou..."
2,“Two things are infinite: the universe and hum...,Albert Einstein,"[ ""human-nature"", ""humor"", ""infinity"", ""ph..."
3,"“So many books, so little time.”",Frank Zappa,"[ ""books"", ""humor""]"
4,“A room without books is like a body without a...,Marcus Tullius Cicero,"[ ""books"", ""simile"", ""soul""]"
...,...,...,...
95,“I'm not afraid of death; I just don't want to...,Woody Allen,"[ ""death"", ""fear"", ""humor"", ""life""]"
96,“If one cannot enjoy reading a book over and o...,Oscar Wilde,"[ ""books"", ""reading""]"
97,"“If you want your children to be intelligent, ...",Albert Einstein,"[ ""children"", ""fairy-tales""]"
98,“Have you fallen in love with the wrong person...,"Cassandra Clare,","[ ""city-of-bones"", ""jace""]"


In [69]:
df.to_excel('updated_quotes.xlsx', index=False)

In [70]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


quote_texts = df['quote'].tolist()
embeddings = model.encode(quote_texts, show_progress_bar=True)


np.save("quote_embeddings.npy", embeddings)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [71]:
import faiss

embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings))
faiss.write_index(index, "quote_index.faiss")


In [72]:
def search_quotes(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        quote = df.iloc[idx]
        results.append({
            "quote": quote["quote"],
            "author": quote["author"],
            "tags": quote["tags"],
            "score": float(dist)
        })
    return results


In [73]:
query = "quotes about imagination by Einstein"
results = search_quotes(query)

for r in results:
    print(f"\"{r['quote']}\"\n— {r['author']} (Tags: {r['tags']})\nScore: {r['score']:.2f}\n")


"“I am enough of an artist to draw freely upon my imagination. Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.”"
— Albert Einstein (Tags: [  "1929",  "imagination",  "inspirational",  "viereck-interview"])
Score: 0.68

"“Everything you can imagine is real.”"
— Pablo Picasso (Tags: [  "art",  "imagination",  "inspirational",  "life"])
Score: 0.87

"“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”"
— Albert Einstein (Tags: [  "human-nature",  "humor",  "infinity",  "philosophy",  "science",  "stupidity",  "universe"])
Score: 0.91

"“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.”"
— Albert Einstein (Tags: [  "children",  "fairy-tales"])
Score: 0.93

"“If you only read the books that everyone else is reading, you can only think what everyone else is thinking.”"
— Haruki Murakami, (Tags: [  "books",  

In [74]:
!pip install sentence-transformers faiss-cpu transformers pandas


In [75]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


gen_model_name = "google/flan-t5-base"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)


def generate_answer(query, retrieved_quotes):

    context = "\n".join([f"\"{q['quote']}\" — {q['author']}" for q in retrieved_quotes])

    prompt = f"Context:\n{context}\n\nQuestion: {query}"

    inputs = gen_tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = gen_model.generate(**inputs, max_new_tokens=200)


    return gen_tokenizer.decode(outputs[0], skip_special_tokens=True)


In [76]:
query = "quotes about imagination by Einstein"


retrieved = search_quotes(query, top_k=5)


answer = generate_answer(query, retrieved)


print(" Generated Answer:\n", answer)
print("\n Retrieved Quotes:")
for r in retrieved:
    print(f"\"{r['quote']}\"\n— {r['author']} (Tags: {r['tags']})\nScore: {r['score']:.2f}\n")


💬 Generated Answer:
 "Everything you can imagine is real."

📚 Retrieved Quotes:
"“I am enough of an artist to draw freely upon my imagination. Imagination is more important than knowledge. Knowledge is limited. Imagination encircles the world.”"
— Albert Einstein (Tags: [  "1929",  "imagination",  "inspirational",  "viereck-interview"])
Score: 0.68

"“Everything you can imagine is real.”"
— Pablo Picasso (Tags: [  "art",  "imagination",  "inspirational",  "life"])
Score: 0.87

"“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”"
— Albert Einstein (Tags: [  "human-nature",  "humor",  "infinity",  "philosophy",  "science",  "stupidity",  "universe"])
Score: 0.91

"“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.”"
— Albert Einstein (Tags: [  "children",  "fairy-tales"])
Score: 0.93

"“If you only read the books that everyone else is reading, you can only

In [77]:
pip install streamlit


In [83]:
import streamlit as st

st.set_page_config(page_title="Quote RAG App", layout="centered")
st.title(" Quote Search & Answering (RAG)")
st.markdown("Ask a natural language question about quotes (e.g., *What did Einstein say about imagination?*)")


query = st.text_input(" Enter your question:")

if query:
    with st.spinner(" Retrieving quotes and generating answer..."):
        retrieved = search_quotes(query)
        answer = generate_answer(query, retrieved)


    st.subheader(" Generated Answer")
    st.write(answer)


    st.subheader(" Retrieved Quotes")
    for r in retrieved:
        st.markdown(f"> *{r['quote']}*  \n— **{r['author']}** (Tags: `{r['tags']}`)")


2025-05-27 09:32:49.316 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 09:32:49.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 09:32:49.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 09:32:49.321 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 09:32:49.322 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 09:32:49.324 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 09:32:49.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 09:32:49.326 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [1]:
!wget -qO- ipv4.icanhazip.com


34.106.75.2


In [2]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 3s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [3]:
!streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.75.2:8501

⠦⠧⠇your url is: https://fast-parts-make.loca.lt
2025-05-27 09:42:55.340856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748338975.373865   79406 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748338975.383457   79406 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-27 09:42:55.413906: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following